# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
# Import cities file as DataFrame
cities = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Barrow,71.2906,-156.7887,33.82,86,90,16.11,US,1632029214
1,Yellowknife,62.4560,-114.3525,43.32,81,75,13.80,CA,1632029107
2,Agropoli,40.3469,14.9966,71.44,78,9,3.74,IT,1632029216
3,Mataura,-46.1927,168.8643,50.49,77,100,1.99,NZ,1632029218
4,Tahoua,14.8888,5.2692,78.78,83,0,4.61,NE,1632029219


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [11]:
# Store latitude and longitude in locations
locations = cities[["Lat", "Lng"]]
humidity = cities["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Creata a DataFrame with ideal weather conditions and drop rows with null values
ideal_weather = cities.loc[(cities["Max Temp"] >= 70) & (cities["Max Temp"] <= 80) &
                                (cities["Wind Speed"] <= 5) & (cities["Cloudiness"] == 0)].dropna()
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Tahoua,14.8888,5.2692,78.78,83,0,4.61,NE,1632029219
41,Kibala,9.1104,18.3463,73.35,94,0,0.00,TD,1632029273
57,Sydney,-33.8679,151.2073,78.62,32,0,1.99,AU,1632029034
122,Mizdah,31.4451,12.9801,70.00,27,0,3.27,LY,1632029393
151,Rosetta,31.4044,30.4164,71.42,58,0,2.01,EG,1632029306
230,Conde,-7.2597,-34.9075,71.06,88,0,4.61,BR,1632029547
307,Porto Velho,-8.7619,-63.9039,73.45,94,0,1.14,BR,1632029664
308,Lajas,18.0500,-67.0593,78.19,82,0,4.00,PR,1632029666
353,Bari,41.1177,16.8512,74.84,80,0,4.61,IT,1632029731


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

In [7]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# Params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # Get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # Convert to json
    hotel_name = hotel_name.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Tahoua,14.8888,5.2692,78.78,83,0,4.61,NE,1632029219,Hotel Etab
1,Kibala,9.1104,18.3463,73.35,94,0,0.00,TD,1632029273,hotel safari
2,Sydney,-33.8679,151.2073,78.62,32,0,1.99,AU,1632029034,Shangri-La Sydney
3,Mizdah,31.4451,12.9801,70.00,27,0,3.27,LY,1632029393,فندق مزدة الكبير
4,Rosetta,31.4044,30.4164,71.42,58,0,2.01,EG,1632029306,Hilton Alexandria Corniche
5,Conde,-7.2597,-34.9075,71.06,88,0,4.61,BR,1632029547,Hotel Nord Luxxor Tambaú
6,Porto Velho,-8.7619,-63.9039,73.45,94,0,1.14,BR,1632029664,Hotel Slaviero Essential Porto Velho
7,Lajas,18.0500,-67.0593,78.19,82,0,4.00,PR,1632029666,Hilton Ponce Golf & Casino Resort
8,Bari,41.1177,16.8512,74.84,80,0,4.61,IT,1632029731,Riva Del Sole Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Create a marker_layer
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))